In [2]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle
from lmfit import Model
from lmfit import models

# Choosing a set of dataframes
pickle_name = 'df_all'

# Choosing a data type
df_type = 'cases'

# Choosing a model. See https://lmfit.github.io/lmfit-py/builtin_models.html
model = models.PowerLawModel()

# Import the dataframe
pickle_file = open(f'../output/pickles/{pickle_name}''.p', 'rb')
df= pickle.load(pickle_file)[df_type]
pickle_file.close()

# Curve fit
df = df.groupby('day').sum().reset_index()
df = df[['day', 'cases']]
params = model.make_params()
result = model.fit(df.cases, params, x=df.day.to_list())
mpl.style.use('dark_background')
result.plot_fit(xlabel='days since 2020-01-22', ylabel='Global cases', datafmt='og', fitfmt='r')

NameError: name 'model' is not defined